In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

import wandb
from wandb.integration.keras import WandbCallback


import carbontracker
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM, Dropout
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

In [20]:
data_dir = 'data/DailyDelhiClimateTrain.csv'
df = pd.read_csv(data_dir)

In [21]:
# Initialize a W&B run and set hyperparameters
wandb.init(
    project="MLOPS-Hackathon",   # Name of your project
    entity="emiliewedenborg-technical-university-of-denmark",  # Replace with your W&B entity
    name="temperature-forecasting-lstm",  # Name of the run

    config={
        "epochs": 10,
        "batch_size": 32,
        "learning_rate": 0.001,
        "optimizer": "adam"
    }
)

config = wandb.config


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


In [22]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace= True)

n_cols = 1
dataset = df["meantemp"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

(1462, 1)

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

In [24]:
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

Train Size : 1096 Test Size : 366


In [25]:
train_data = scaled_data[0:train_size, :]
train_data.shape

(1096, 1)

In [26]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(scaled_data)):
    x_train.append(scaled_data[i-time_steps:i, :n_cols])
    y_train.append(scaled_data[i, :n_cols])

In [27]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

In [28]:
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))

In [29]:
x_train.shape , y_train.shape

((1402, 60, 1), (1402, 1))

In [30]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= ["mean_absolute_error"])

c:\Users\aejew\OneDrive - Danmarks Tekniske Universitet\Dokumenter\GitHub\MLOPS-Hackathon\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,397 (497.65 KB)

 Trainable params: 42,465 (165.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 84,932 (331.77 KB)

In [36]:
# Train with W&B callback

history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    callbacks=[
        WandbCallback(
            training_data = (x_train, y_train),  # Training data for logging
            save_model=True,       # saves model weights
            log_weights=True,      # log weights
            log_gradients=True,    # log gradients
            save_graph=False       # ✅ disable graph logging
        )
    ]
)

ImportError: `keras.optimizers.legacy` is not supported in Keras 3. When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy` optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable `TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

In [40]:
# Fitting the LSTM to the Training set
history = model.fit(x_train, y_train, epochs= 100, batch_size= 32,callbacks=[WandbCallback()])

Epoch 1/100


AttributeError: 'Node' object has no attribute 'inbound_layers'

In [ ]:
history = model.fit(
    x_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    callbacks=[WandbCallback(save_graph=False)]
)


Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - loss: 0.0162 - mean_absolute_error: 0.0991
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0057 - mean_absolute_error: 0.0600
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.0051 - mean_absolute_error: 0.0564
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0048 - mean_absolute_error: 0.0546
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 82ms/step - loss: 0.0050 - mean_absolute_error: 0.0557
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0046 - mean_absolute_error: 0.0530
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - loss: 0.0044 - mean_absolute_error: 0.0521
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - loss: 0.0045 - mean_absolute_error: 0.0526
Epoch 9/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 137ms/step - loss: 0.0047 - mean_absolute_error: 0.0540
Epoch 10/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - loss: 0.0044 - mean_absolute_error: 0.0523
Epoch 11/100
44/44 ━━━━━━━━

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

NameError: name 'history' is not defined

<Figure size 1200x800 with 0 Axes>